# MODELO DE CLASIFICACIÓN DE LA COLUMNA 'cut' DEL DATASET 'DIAMONDS'.

## 1.-CARGA DE LIBRERÍAS.

In [1]:
# Librería básica de tratamiento numérico.
import numpy as np

# Librería básica de tratamiendo de DataFrames.
import pandas as pd

# Librerías de Pipelines.
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer

# Liberías de Pre-Procesamiento.
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, OrdinalEncoder, LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer

# Librerías de modelado.
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

# Librerías de evaluación de modelos.
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Librería de optimización de parámetros.
from sklearn.model_selection import GridSearchCV

# Librería de exportación de modelos.
import joblib

## 2.-CARGA DEL DATASET.

In [2]:
ruta = 'https://raw.githubusercontent.com/mwaskom/seaborn-data/refs/heads/master/'
archivo = 'diamonds.csv'
url = ruta + archivo
df = pd.read_csv(url)

## 3.-COMPROBACIÓN DEL DATASET.

<p>Dado que la variable 'price' es una variable dependiente de todas las demás variables del DataFrame y, para la predicción de la variable 'cut', no es aplicable, se elimina del DataFrame la variable 'price'.</p>

In [3]:
df = df.drop(columns=['price'])

### 3.1.-Cabecera del DataSet.

In [4]:
df.head()

,carat,cut,color,clarity,depth,table,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,4.34,4.35,2.75


### 3.2.-Tamaño del DataSet.

In [5]:
num_filas = df.shape[0]
num_columnas = df.shape[1]
print(f'El DataSet cargado tiene {num_filas} filas y {num_columnas} columnas.')

El DataSet cargado tiene 53940 filas y 9 columnas.


### 3.3.-Información del DataSet.

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53940 non-null  float64
 1   cut      53940 non-null  object 
 2   color    53940 non-null  object 
 3   clarity  53940 non-null  object 
 4   depth    53940 non-null  float64
 5   table    53940 non-null  float64
 6   x        53940 non-null  float64
 7   y        53940 non-null  float64
 8   z        53940 non-null  float64
dtypes: float64(6), object(3)
memory usage: 3.7+ MB


### 3.4.-Descripción del DataSet.

In [7]:
df.describe(include='all')

,carat,cut,color,clarity,depth,table,x,y,z
count,53940.000000,53940,53940,53940,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
unique,NaN,5,7,8,NaN,NaN,NaN,NaN,NaN
top,NaN,Ideal,G,SI1,NaN,NaN,NaN,NaN,NaN
freq,NaN,21551,11292,13065,NaN,NaN,NaN,NaN,NaN
mean,0.797940,NaN,NaN,NaN,61.749405,57.457184,5.731157,5.734526,3.538734
std,0.474011,NaN,NaN,NaN,1.432621,2.234491,1.121761,1.142135,0.705699
min,0.200000,NaN,NaN,NaN,43.000000,43.000000,0.000000,0.000000,0.000000
25%,0.400000,NaN,NaN,NaN,61.000000,56.000000,4.710000,4.720000,2.910000
50%,0.700000,NaN,NaN,NaN,61.800000,57.000000,5.700000,5.710000,3.530000
75%,1.040000,NaN,NaN,NaN,62.500000,59.000000,6.540000,6.540000,4.040000


### 3.4.-Filas con valores '0'.

In [8]:
filas_con_cero = df[df.isin([0, 0.0, '0']).any(axis=1)]
filas_con_cero.shape[0]-1

19

Hay 19 filas con algún valor 0 en alguna de sus columnas.

Sustituimos los valores 0 o 0.0 por la mediana.

In [9]:
for col in df.select_dtypes(include=np.number).columns:
    mediana = df[col].median()
    df.loc[df[col].isin([0, 0.0]), col] = mediana

In [10]:
filas_con_cero = df[df.isin([0, 0.0, '0']).any(axis=1)]
filas_con_cero

,carat,cut,color,clarity,depth,table,x,y,z


Ya no hay celdas con valores 0.

## 4.-COLUMNA A PREDECIR.

In [11]:
col_predict = 'cut'
col_predict_encoded = col_predict + '_encoded' 

### 4.1.-Encoding de la columna a predecir.

Dado que la columna categórica a predecir, 'cut', es además de tipo ordinal, es preciso registrar los valores posibles ordenados que puede adoptar la variable. Este paso se realiza mediante el mapeo del orden de los valores. Este mapeo del LabelEncoder se ha de exportar también para que posteriormente, desde Streamlit, se pueda recuperar y utilizar para mostrar la predicción de forma correcta.

In [12]:
orden_predict = ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']   # Se mantiene el orden de los valores posibles de la variable a predecir.
encoding_predict = LabelEncoder()
encoding_predict.classes_ = np.array(orden_predict)                 # Se almacenan los valores posibles ordenados de la variable a predecir. 
df[col_predict_encoded] = encoding_predict.fit_transform(df[col_predict])

## 5.-MODELADO DE DATOS.

### 5.1.-Definición de características y variable objetivo.

In [13]:
X = df.drop([col_predict, col_predict_encoded], axis=1)
y = df[col_predict_encoded]

### 5.2.-Definición de datos de prueba y test.

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

### 5.3.-División de tipos de columnas.

In [15]:
# Se crea una lista conteniendo los nombre de las columnas con valores numéricos.
col_num = X.select_dtypes(include=[np.number]).columns.tolist()
print(col_num)

['carat', 'depth', 'table', 'x', 'y', 'z']


In [16]:
# Se crea una lista conteniendo los nombre de las columnas con valores categóricos.
col_cat = X.select_dtypes(exclude=[np.number]).columns.tolist()
print(col_cat)

['color', 'clarity']


### 5.4.-Generación de Pipelines.

#### 5.4.1.-Pipelines de Pre-Procesado.

##### 5.4.1.1.-Pipelines para variables numéricas.

In [17]:
pipeline_num = Pipeline(steps = [
    ('imputer', IterativeImputer(random_state=42)),
    ('scaler', RobustScaler())
])

##### 5.4.1.2.-Pipelines para variables categóricas.

En este caso, existen columnas categóricas de tipo ordinal: 'cut', 'color' y 'clarity'. Se ha de realizar el encoding apropiadamente (OrdinalEncoder), de acuerdo a su orden establecido. 

In [18]:
# Valores posibles de la columna 'color'. Ordenados de menor a mayor calidad.
orden_color = ['D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

# Valores posibles de la columna 'clarity'. Ordenados de menor a mayor calidad.
orden_clarity = ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF', 'FL']

Se definen las listas de las columnas ordinales.

In [19]:
orden_categorias = [orden_color, orden_clarity]

In [20]:
pipeline_ordinal_cat = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder(categories=orden_categorias, dtype = np.int64))
])

##### 5.4.1.3.-Combinación de Pipelines.

In [21]:
# Se crea el ColumnTransformer para combinar todos los pipelines del pre-procesado.
preprocesador = ColumnTransformer(
    transformers=[
        ('columnas_num', pipeline_num, col_num),
        ('columnas_cat', pipeline_ordinal_cat, col_cat)
    ],
    remainder='drop'  # Se descartan las columnas que no se han especificado.
)

#### 5.4.2.-Selección del modelo.

##### 5.4.2.1-Comparativa de modelos.

In [22]:
# Algoritmos que se van a evaluar.
modelos = {
    "LogisticRegression": LogisticRegression(max_iter=1000, random_state=42),
    "RandomForestClassifier": RandomForestClassifier(random_state=42),
    "KNeighborsClassifier": KNeighborsClassifier()
}

resultados = []                                         # Lista de almacenaje de resultados de las métricas de cada algoritmo.
for nombre, modelo in modelos.items():
    pipeline_modelo = Pipeline([
        ('preprocesador', preprocesador),               # Se aplica el preprocesado.
        ('modelo', modelo)
    ])
    pipeline_modelo.fit(X_train, y_train)               # Se entrena el modelo con los datos de entrenamiento.
    predicciones = pipeline_modelo.predict(X_test)      # Se realiza la predicción con los datos de prueba.
    
    accuracy = accuracy_score(y_test, predicciones)
    precision = precision_score(y_test, predicciones, average='weighted')
    recall = recall_score(y_test, predicciones, average='weighted')
    f1 = f1_score(y_test, predicciones, average='weighted')
    
    resultados.append({"Modelo": nombre, "Accuracy": accuracy, "Precision": precision, "Recall": recall, "F1": f1})

df_resultados = pd.DataFrame(resultados)
df_resultados

,Modelo,Accuracy,Precision,Recall,F1
0,LogisticRegression,0.652670,0.638976,0.652670,0.626963
1,RandomForestClassifier,0.773637,0.766355,0.773637,0.765869
2,KNeighborsClassifier,0.665554,0.638665,0.665554,0.642514


El algoritmo RandomForestClassifier muestra los mejores resultados de todos los algoritmos probados.

##### 5.4.2.2-Optimización de parámetros del modelo seleccionado.

In [23]:
modelo_seleccionado = make_pipeline(preprocesador, RandomForestClassifier(random_state=42))
parametros_grid = {
    'randomforestclassifier__n_estimators': [50, 100, 200],
    'randomforestclassifier__max_depth': [None, 10, 20]
}

grid = GridSearchCV(modelo_seleccionado, parametros_grid, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)
print("Mejores parámetros:", grid.best_params_)

Mejores parámetros: {'randomforestclassifier__max_depth': None, 'randomforestclassifier__n_estimators': 200}


In [24]:
pipeline_final = grid.best_estimator_

### 5.5.-Exportación del modelo y del encoder.

In [25]:
# Se exporta el modelo de clasificación
joblib.dump(pipeline_final, '../Models/pipeline_clasificacion.joblib')

# Se exporta el encoder, con los valores posibles de la variable a predecir.
joblib.dump(encoding_predict, '../Models/label_encoder_clasificacion.joblib')

['../Models/label_encoder_clasificacion.joblib']